## Step 1: Import the required packages

- I will not be using SDKs, mostly using the standard packages
- Using python-dotenv to load the environment variables

In [5]:
import os
import common
import json
import datetime
from dotenv import load_dotenv


## Step 2: Load the environment variables to make calls to the GPT endpoint

In [6]:
load_dotenv()
api_full_uri = os.getenv("OPENAI_FULL_URI")
api_key = os.getenv("OPENAI_KEY")
email_URI = os.getenv("EMAIL_URI")

common.uri = api_full_uri
common.key = api_key

## Step 3: Send a simple Prompt via a POST request for Completion

- I just want to show that I'm not using any SDKs (no OpenAI, no Semantic Kernel, no LangChain, etc.)

In [7]:
payload = {
    "messages": [
        {
            "role": "user",
            "content": "What is the speed of light?"
        }
    
    ],
    "max_tokens": 100,
    "temperature": 0.2
}
json_data = common.send_post(api_full_uri, api_key, payload)
json_data["choices"][0]["message"]["content"]

'The speed of light in a vacuum is exactly 299,792,458 meters per second (approximately 186,282 miles per second). This speed is considered a fundamental constant of nature and is denoted by the letter "c" in physics equations, most famously in Einstein\'s equation E=mc², which relates energy (E) to mass (m) with the speed of light (c) as the proportionality constant.'

## Step 4: Read the contents of the `data\portfolio.csv` file

In [8]:
def read_file(filePath:str)->str:
    with open(filePath, 'r') as file:
        # Read the contents of the file
        return file.read()
file_contents = read_file("data/portfolio.csv")

## Step 5: Send a more complex Prompt to extract the ticker, cost, and quantity from the portfolio file

In [9]:
prompt_template = """system:
You are a financial assistant that can help provide investment information from a user's investment portfolio.
If the question in not related to managing financial investments, say, "Please contact support for more assistance."

user:
For the following portfolio, extract the ticker symbol, price, and quantity:

Portfolio:
""""""
<FILE_CONTENTS>
""""""

JSON Output format:
""""""
[
    { "ticker": "AAPL", "price": 146.92, "quantity": 10 },
    { "ticker": "MSFT", "price": 412.92, "quantity": 50 }
]
""""""

Provide the answer in JSON format only. DO NOT provide any comments, titles or explanations. 
"""

prompt = prompt_template.replace("<FILE_CONTENTS>", file_contents)

print(prompt)

json_response = common.process_completion(prompt,500,0.2)
print("assistant:")
print(json_response)

json_data = json.loads(json_response) # Simple Validation

system:
You are a financial assistant that can help provide investment information from a user's investment portfolio.
If the question in not related to managing financial investments, say, "Please contact support for more assistance."

user:
For the following portfolio, extract the ticker symbol, price, and quantity:

Portfolio:

Symbol,Average_Cost,QTY
MSFT,200,300
AAPL,114,200
AMZN,125,50
TSLA,900,100
NFLX,540,80
NVDA,450,50


JSON Output format:

[
    { "ticker": "AAPL", "price": 146.92, "quantity": 10 },
    { "ticker": "MSFT", "price": 412.92, "quantity": 50 }
]


Provide the answer in JSON format only. DO NOT provide any comments, titles or explanations. 

assistant:
[
    { "ticker": "MSFT", "price": 200, "quantity": 300 },
    { "ticker": "AAPL", "price": 114, "quantity": 200 },
    { "ticker": "AMZN", "price": 125, "quantity": 50 },
    { "ticker": "TSLA", "price": 900, "quantity": 100 },
    { "ticker": "NFLX", "price": 540, "quantity": 80 },
    { "ticker": "NVDA", "price"

## Getting the latest stock prices, calculate the portfolio value and generate and HTML formatted report

In [10]:
def calculate_and_generate_table(json_data):
    portfolio_value = 0.0
    rows = []
    rows.append("<h2>Portfolio Report</h2>")
    now_str = datetime.datetime.now().strftime("%x %X")
    rows.append(f"<p>Date: {now_str}</p>")
    rows.append("<table style='width:100%'>\n<tr><th>Ticker</th><th>Price</th><th>Quantity</th><th>Cost Basis</th><th>Current Value</th><th>Gain/Loss</th></tr>")
    high = 0
    high_ticker = ''
    low=0
    low_ticker = ''
    for item in json_data:
        #print(item)        
        latest_price = round(common.get_stock_price(item["ticker"]),2)
        cost_basis = round(item["price"] * item["quantity"],2)
        current_value = round(latest_price * item["quantity"],2)
        portfolio_value += current_value
        gain_loss = round(current_value - cost_basis,2)
        if gain_loss > high:
            high = gain_loss
            high_ticker = item["ticker"]
        if gain_loss < low:
            low = gain_loss
            low_ticker = item["ticker"]
        #print("Ticker: ", item["ticker"], "Price: ", latest_price, "Quantity: ", item["quantity"], "Cost Basis: ", cost_basis, "Current Value: ", current_value, "Gain/Loss: ", gain_loss)
        rows.append(f"<tr><td>{item['ticker']}</td><td>{latest_price}</td><td>{item['quantity']}</td><td>{cost_basis}</td><td>{current_value}</td><td>{gain_loss}</td></tr>")
    rows.append("</table>")
    rows.append(f"<h3>Portfolio Value: {portfolio_value}</h3>")
    rows.append(f"<h3>Highest Gain: {high_ticker} {high}</h3>")
    rows.append(f"<h3>Lowest Gain: {low_ticker} {low}</h3>")
    return '\n'.join(rows)

table = calculate_and_generate_table(json_data)
print(table)
    

<h2>Portfolio Report</h2>
<p>Date: 02/10/24 17:24:23</p>
<table style='width:100%'>
<tr><th>Ticker</th><th>Price</th><th>Quantity</th><th>Cost Basis</th><th>Current Value</th><th>Gain/Loss</th></tr>
<tr><td>MSFT</td><td>420.55</td><td>300</td><td>60000</td><td>126165.0</td><td>66165.0</td></tr>
<tr><td>AAPL</td><td>188.85</td><td>200</td><td>22800</td><td>37770.0</td><td>14970.0</td></tr>
<tr><td>AMZN</td><td>174.45</td><td>50</td><td>6250</td><td>8722.5</td><td>2472.5</td></tr>
<tr><td>TSLA</td><td>193.57</td><td>100</td><td>90000</td><td>19357.0</td><td>-70643.0</td></tr>
<tr><td>NFLX</td><td>561.32</td><td>80</td><td>43200</td><td>44905.6</td><td>1705.6</td></tr>
<tr><td>NVDA</td><td>721.33</td><td>50</td><td>22500</td><td>36066.5</td><td>13566.5</td></tr>
</table>
<h3>Portfolio Value: 272986.6</h3>
<h3>Highest Gain: MSFT 66165.0</h3>
<h3>Lowest Gain: TSLA -70643.0</h3>


## Step 6: Send an HTML formatted report

In [11]:
common.send_logic_apps_email(email_URI,"alemor@microsoft.com",table)

Email sent to: alemor@microsoft.com
